In [16]:
import mlflow
from azureml.core import Workspace
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder


In [3]:
print(mlflow.__version__)


2.8.1


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
ws = Workspace.from_config() 

#### Mettre en place un URI de Tracking des experimentation MLflow sur Azure

Nous indiquons à mlflow que l'URI de tracking est celui de mlflow dans azure ML. Toutefois, le lien de tracking de Mlflow dans azure à la forme suivante:

azureml://<region>.api.azureml.ms/mlflow/v1.0/subscriptions/<subscription-id>/resourceGroups/<resource-group>/providers/Microsoft.MachineLearningServices/workspaces/<aml-workspace>?

In [6]:
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

In [7]:
mlflow.get_tracking_uri()

'azureml://francecentral.api.azureml.ms/mlflow/v2.0/subscriptions/2c2d1cc7-f0fc-4174-b403-4bdd35238850/resourceGroups/Learn_MLOps/providers/Microsoft.MachineLearningServices/workspaces/Ewins_ML?'

In [8]:
experiment_name = 'student-mental'
mlflow.set_experiment(experiment_name)

2023/12/02 17:43:48 INFO mlflow.tracking.fluent: Experiment with name 'student-mental' does not exist. Creating a new experiment.


<Experiment: artifact_location='', creation_time=1701535429720, experiment_id='56602126-7e26-4fb9-b6eb-b70021ff9ef5', last_update_time=None, lifecycle_stage='active', name='student-mental', tags={}>

#### chargement du dataset et traitements necéssaires 

In [9]:
df = pd.read_csv("../data/Student Mental health.csv")

In [10]:
df.shape

(101, 11)

In [12]:
data = df.dropna(how='any',axis=0) 
data.shape

(100, 11)

In [13]:
data.rename(columns = {'Choose your gender':'Choose_your_gender',
                     'What is your course?':'What_is_your_course', 
                     'Your current year of Study':'Your_current_year_of_Study',
                     'What is your CGPA?':'What_is_your_CGPA', 
                     'Marital status':'Marital_status', 
                     'Do you have Depression?':'Depression', 
                     'Do you have Anxiety?':'Do_you_have_Anxiety',
                     'Do you have Panic attack?':'Do_you_have_Panic_attack', 
                     'Did you seek any specialist for a treatment?':'Did_you_seek_any_specialist_for_a_treatment'}, inplace = True)
data.head()

,Timestamp,Choose_your_gender,Age,What_is_your_course,Your_current_year_of_Study,What_is_your_CGPA,Marital_status,Depression,Do_you_have_Anxiety,Do_you_have_Panic_attack,Did_you_seek_any_specialist_for_a_treatment
0,8/7/2020 12:02,Female,18.0,Engineering,year 1,3.00 - 3.49,No,Yes,No,Yes,No
1,8/7/2020 12:04,Male,21.0,Islamic education,year 2,3.00 - 3.49,No,No,Yes,No,No
2,8/7/2020 12:05,Male,19.0,BIT,Year 1,3.00 - 3.49,No,Yes,Yes,Yes,No
3,8/7/2020 12:06,Female,22.0,Laws,year 3,3.00 - 3.49,Yes,Yes,No,No,No
4,8/7/2020 12:13,Male,23.0,Mathemathics,year 4,3.00 - 3.49,No,No,No,No,No


In [14]:
data["Depression"].replace({"Yes": 1, "No": 0}, inplace=True)
data["Marital_status"].replace({"Yes": 1, "No": 0}, inplace=True)
data["Do_you_have_Anxiety"].replace({"Yes": 1, "No": 0}, inplace=True)
data["Do_you_have_Panic_attack"].replace({"Yes": 1, "No": 0}, inplace=True)
data["Did_you_seek_any_specialist_for_a_treatment"].replace({"Yes": 1, "No": 0}, inplace=True)
data["Choose_your_gender"].replace({"Male": 1, "Female": 0}, inplace=True)
data["What_is_your_course"].replace({
                                     "Islamic education": "Islamic Education",
                                     "Law": "Laws",
                                     "Pendidikan islam": "Pendidikan Islam",
                                     "Pendidikan Islam ": "Pendidikan Islam",
                                     "psychology": "Psychology",
                                     "koe": "KOE",
                                     "Koe": "KOE",
                                     "Kirkhs": "KIRKHS",
                                     "Benl": "BENL",
                                     "Nursing ": "Diploma Nursing",
                                     "engin": "Engineering",
                                     "Engine": "Engineering"}, inplace=True)

data["What_is_your_CGPA"].replace({"3.50 - 4.00 ": "3.50 - 4.00"}, inplace=True)

data["Your_current_year_of_Study"].replace({
                                     "Year 1": 1,
                                     "year 1": 1,
                                     "year 2": 2,
                                     "Year 2": 2,
                                     "year 3": 3,
                                     "Year 3": 3,
                                     "year 4": 4,
                                     "Year 4": 4,}, inplace=True)


In [17]:
le = LabelEncoder()
data['course_encoded'] = le.fit_transform(data['What_is_your_course'])

In [18]:
le2 = LabelEncoder()
data['CGPA_encoded'] = le2.fit_transform(data['What_is_your_CGPA'])

In [19]:
data.drop(columns=["Timestamp","What_is_your_course","What_is_your_CGPA"],inplace=True)

In [20]:
X = data.drop(columns=["Depression"])
y = data.Depression

In [21]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
df_scaled=scaler.fit_transform(X)

In [23]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df_scaled,y,test_size=0.2,random_state=0)

In [24]:
print("X_train:",x_train.shape)
print("X_test:",x_test.shape)
print("Y_train:",y_train.shape)
print("Y_test:",y_test.shape)

X_train: (80, 9)
X_test: (20, 9)
Y_train: (80,)
Y_test: (20,)


In [28]:
rf_clf = RandomForestClassifier() 

In [25]:
mlflow.sklearn.autolog(max_tuning_runs=None)

#### Hyperparameter tuning 

Nous alons utiliser l'algorithme GridSearchCV pour trouver les meilleurs paramétres

In [26]:
param_grid = {'n_estimators': [10,20,30], 'max_depth':[2,7,10]}

In [29]:
gridcv=GridSearchCV(rf_clf,param_grid=param_grid,scoring = ['roc_auc', 'precision', 'recall', 'f1', 'accuracy'],refit = 'roc_auc',cv=10)

In [30]:
gridcv.fit(x_train, y_train)

2023/12/02 17:54:20 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'aadcdddd-5811-41b5-88a8-3b50625ce66b', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2023/12/02 17:54:20 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [2, 7, 10], 'n_estimators': [10, 20, 30]},
             refit='roc_auc',
             scoring=['roc_auc', 'precision', 'recall', 'f1', 'accuracy'])

## 2.  L'enregistrement du modèle 
L'enregistrement du modèle dans le registre de modèles d'Azure Machine Learning a pour but de permettre aux utilisateurs de suivre les modifications apportées au modèle via la gestion des versions du modèle.

In [31]:
from azureml.core import Experiment, Workspace
#experiment_name = 'diabetes-sklearn'
experiment = Experiment(ws, experiment_name)

In [32]:
# afficher tous les runs 
for r in experiment.get_runs():
    print(r)

Run(Experiment: student-mental,
Id: aadcdddd-5811-41b5-88a8-3b50625ce66b,
Type: None,
Status: Completed)


Trouver le Run actuel 

In [33]:
run_id = 'aadcdddd-5811-41b5-88a8-3b50625ce66b'
run = [r for r in experiment.get_runs() if r.id == run_id][0]

In [34]:
model = run.register_model(model_name = 'student_mental_model', model_path = 'best_estimator/model.pkl')

## 4. Déploiement en local

Maintenant nous allons debuger le web service locallement avant de le déployer en production avec ACI " Azure container Instance"

In [35]:
from azureml.core.model import Model
model = Model(ws, 'student_mental_model', version=1)

In [54]:
from azureml.core import Environment
env = Environment.from_conda_specification(name='student-mental-env', file_path="./conda.yaml")
env.register(ws)

{
    "assetId": "azureml://locations/francecentral/workspaces/3252fc44-3209-481d-bf47-5f2f1c75eb85/environments/student-mental-env/versions/4",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20231030.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "student-ment

### Définons la configuration d'inférence 

In [55]:
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(
    environment=env,
    source_directory=".",
    entry_script="./score.py",
)

Warning, azureml-defaults not detected in provided environment pip dependencies. The azureml-defaults package contains requirements for the inference stack to run, and should be included.


### Définons la configuration de déploiement

In [38]:
from azureml.core.webservice import LocalWebservice
deployment_config = LocalWebservice.deploy_configuration(port=6789)

### Déployons le service localement

Avant d'exécuter la cellule ci-dessous, assurez vous que Docker est démarré 

In [40]:
service = Model.deploy(
    workspace = ws,
    name = 'student-mental-prediction',
    models = [model],
    inference_config = inference_config,
    deployment_config = deployment_config,
    overwrite=True)

Generating Docker build context.
2023/12/02 17:31:59 Downloading source code...
2023/12/02 17:31:59 Finished downloading source code
2023/12/02 17:32:00 Creating Docker network: acb_default_network, driver: 'bridge'
2023/12/02 17:32:00 Successfully set up Docker network: acb_default_network
2023/12/02 17:32:00 Setting up Docker configuration...
2023/12/02 17:32:00 Successfully set up Docker configuration
2023/12/02 17:32:00 Logging in to registry: 3252fc443209481dbf475f2f1c75eb85.azurecr.io
2023/12/02 17:32:01 Successfully logged into 3252fc443209481dbf475f2f1c75eb85.azurecr.io
2023/12/02 17:32:02 Volume source scriptsFromEms successfully created
2023/12/02 17:32:02 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2023/12/02 17:32:02 Scanning for dependencies...
2023/12/02 17:32:02 Successfully scanned dependencies
2023/12/02 17:32:02 Launching container with name: acb_step_0
Sending build context to Docker daemon  77.31kB
Step 1/

Removing intermediate container 232be339a22a
 ---> c5a20bc9c99e
Step 9/21 : ENV PATH /azureml-envs/azureml_d6018e400accfe4dea138b181d8a1018/bin:$PATH
 ---> Running in 92d1289aa2fa
Removing intermediate container 92d1289aa2fa
 ---> 60a30a65c4b7
Step 10/21 : COPY azureml-environment-setup/send_conda_dependencies.py azureml-environment-setup/send_conda_dependencies.py
 ---> 0ea1395bcb97
Step 11/21 : RUN echo "Copying environment context"
 ---> Running in 71607d606748
Copying environment context
Removing intermediate container 71607d606748
 ---> 7a5fd0c63c74
Step 12/21 : COPY azureml-environment-setup/environment_context.json azureml-environment-setup/environment_context.json
 ---> 5ec3b1acda98
Step 13/21 : RUN python /azureml-environment-setup/send_conda_dependencies.py -p /azureml-envs/azureml_d6018e400accfe4dea138b181d8a1018
 ---> Running in 996577eefb36
Report materialized dependencies for the environment
Reading environment context
Exporting conda environment
Sending request with mate

Package creation Succeeded
Logging into Docker registry 3252fc443209481dbf475f2f1c75eb85.azurecr.io
Logging into Docker registry 3252fc443209481dbf475f2f1c75eb85.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 3252fc443209481dbf475f2f1c75eb85.azurecr.io/azureml/azureml_c4626ca88f2a0a918c1f760c71f602c2
 ---> ed359519b4d1
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> ba5f48ea0772
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjJjMmQxY2M3LWYwZmMtNDE3NC1iNDAzLTRiZGQzNTIzODg1MCIsInJlc291cmNlR3JvdXBOYW1lIjoibGVhcm5fbWxvcHMiLCJhY2NvdW50TmFtZSI6ImV3aW5zX21sIiwid29ya3NwYWNlSWQiOiIzMjUyZmM0NC0zMjA5LTQ4MWQtYmY0Ny01ZjJmMWM3NWViODUifSwibW9kZWxzIjp7fSwibW9kZWxzSW5mbyI6e319 | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in c86f01e1cc38
 ---> 88b670e88645
Step 4/5 : RUN mv '/var/azureml-app/tmp22l2lmzo.py' /var/azureml-app/main.py
 ---> Running in ffbae9e3b5ab
 ---> ec346732c191
Step 5/5 : CMD ["run

In [69]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
SucceededACI service creation operation finished, operation "Succeeded"


Le fichier model.pkl sera téléchargé à partir d'Azure Machine Learning dans un dossier local temporaire et une image Docker avec les dépendances est créée et enregistrée dans Azure Container Registry (ACR). L'image sera téléchargée d'ACR sur la machine locale et un conteneur Docker exécutant le service Web est construit à partir de l'image localement.



Pour avoir l'URI de scoring lancez la commande suivante 

In [42]:
print (service.scoring_uri)

http://localhost:6789/score


## 6. Déplyons le service sur ACI

In [57]:
from azureml.core import Workspace
ws = Workspace.from_config()
from azureml.core.model import Model
model = Model(ws, 'student_mental_model', version=1)
from azureml.core import Environment
env = Environment.get(workspace = ws, name = 'student-mental-env', version = 1)

In [58]:
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(
    environment=env,
    source_directory=".",
    entry_script="./score.py")

Warning, azureml-defaults not detected in provided environment pip dependencies. The azureml-defaults package contains requirements for the inference stack to run, and should be included.


In [47]:
from azureml.core.webservice import AciWebservice
deployment_config = AciWebservice.deploy_configuration(cpu_cores=0.1, memory_gb=0.5, auth_enabled=False)

In [ ]:
service = Model.deploy(
    workspace = ws,
    name = 'student-mental-service',
    models = [model],
    inference_config = inference_config,
    deployment_config = deployment_config,
    overwrite=True)
    
service.wait_for_deployment(show_output=True)